# Segment, localise and track


In [5]:
from macrohet import dataio, tile, notify
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
import btrack 
import torch
import os
import dask.array as da
import glob
import zarr
import logging
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

# # defining personal trained cellpose model to use
# model_path = '/home/dayn/analysis/models/cellpose/PS0000/macrohet_seg'
# model = models.CellposeModel(gpu=True, 
#                              pretrained_model=model_path)

# ORRRR test the new cellpose model
model = models.Cellpose(gpu=True, model_type='cyto3')

# Initialize the logging configuration
log_dir = "logs"  # Specify the directory where logs will be saved
os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

# Add a FileHandler to save logs to a file in the specified directory
log_file = os.path.join(log_dir, "assay_processing.log")
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s [%(levelname)s]: %(message)s")
file_handler.setFormatter(formatter)
logging.getLogger().addHandler(file_handler)

# Define a function to log progress and potential errors
def log_progress(position, message):
    logging.info(f"Position {position}: {message}")


INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>> cyto3 << model set to be used


Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.1 GB


INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


### Define functions to tidy up main block of code

In [6]:
# define thresholds
segment_size_thresh = 5000
Mtb_load_thresh = 480

# define tracking scale factor
scale_factor = 1/5.04

# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]

# define tracker config fn to use, using a prob_not_assign = 0.1
config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'
# define tracker config fn to use
# config_fn = '/home/dayn/analysis/btrack/models/particle_config.json'

def segment(frame, model = model, channels = [0,0], diameter = 350, #250 #325
            min_size = 5000, model_type = 'pretrained'
           ):
    
    if model_type == 'pretrained':
        
        masks, flows, styles, diams = model.eval(frame, # for default models
                                                 channels = channels, 
                                                 diameter = diameter, 
                                                 min_size = min_size, 
                                                 )
        
        
    else:

        masks, flows, styles = model.eval(frame, # for personal model
                                          channels = channels, 
                                          diameter = diameter, 
                                          min_size = min_size, 
                                          )
    return masks


def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(segmentation=masks,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
                                                   
    return objects


def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]*scale_factor), (0, masks.shape[-1]*scale_factor))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks


def otsu_threshold_stack(images):
    """
    Function to characterise intra-Mφ Mtb load
    Computes Otsu's threshold value and returns a binary segmentation for
    each image in a time series of grayscale images.

    Parameters:
    -----------
    images : ndarray
        A 3D array of shape (n_images, height, width) containing a time series
        of grayscale images.

    Returns:
    --------
    ndarray
        A boolean array of shape (n_images, height, width) containing the
        binary segmentation for each image in the time series.
    """
    segmentations = np.zeros(images.shape, dtype=bool)
    for i, image in tqdm(enumerate(images), 
                         total=len(images), 
                         leave=False, 
                         desc='Otsu segmenting'):
        loaded_image = image.compute().compute()
        threshold = threshold_otsu(loaded_image)
        segmentations[i] = loaded_image > threshold
        
    return segmentations

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [7]:
base_dir = '/mnt/SYNO/macrohet_syno/ND0003/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0301K1F1P1R1,Ok,r03c01f01p01-ch1sk1fk1fl1.tiff,3,1,1,1,0,1,1,...,0,0.135256499,0,2024-02-16T17:15:25.597+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
1,0301K1F1P1R2,Ok,r03c01f01p01-ch2sk1fk1fl1.tiff,3,1,1,1,0,2,1,...,0,0.135256499,0,2024-02-16T17:15:25.813+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
2,0301K1F1P2R1,Ok,r03c01f01p02-ch1sk1fk1fl1.tiff,3,1,1,2,0,1,1,...,2E-06,0.1352586,0,2024-02-16T17:15:26.157+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
3,0301K1F1P2R2,Ok,r03c01f01p02-ch2sk1fk1fl1.tiff,3,1,1,2,0,2,1,...,2E-06,0.1352586,0,2024-02-16T17:15:26.39+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
4,0301K1F1P3R1,Ok,r03c01f01p03-ch1sk1fk1fl1.tiff,3,1,1,3,0,1,1,...,4E-06,0.135260597,0,2024-02-16T17:15:26.733+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349267,0612K154F9P1R2,Ok,r06c12f09p01-ch2sk154fk1fl1.tiff,6,12,9,1,153,2,1,...,0,0.135008901,275402.773,2024-02-19T21:59:46.84+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349268,0612K154F9P2R1,Ok,r06c12f09p02-ch1sk154fk1fl1.tiff,6,12,9,2,153,1,1,...,2E-06,0.135010898,275402.773,2024-02-19T21:59:47.183+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349269,0612K154F9P2R2,Ok,r06c12f09p02-ch2sk154fk1fl1.tiff,6,12,9,2,153,2,1,...,2E-06,0.135010898,275402.773,2024-02-19T21:59:47.4+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349270,0612K154F9P3R1,Ok,r06c12f09p03-ch1sk154fk1fl1.tiff,6,12,9,3,153,1,1,...,4E-06,0.135012895,275402.773,2024-02-19T21:59:47.743+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [8]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC  Replicate #
Row Column                                                           
3   1         UNI     CTRL             0             EC0            1
    2         UNI     CTRL             0             EC0            2
    3          WT     CTRL             0             EC0            1
    4          WT     CTRL             0             EC0            2
    5          WT      PZA            60            EC50            1
    6          WT      PZA            60            EC50            2
    7          WT      RIF           0.1            EC50            1
    8          WT      RIF           0.1            EC50            2
    9          WT      INH          0.04            EC50            1
    10         WT      INH          0.04            EC50            2
    11         WT      BDQ          0.02            EC50            1
    12         WT      BDQ          0.02            EC50            2
4   3          WT     CTRL             0             EC0            3
    4          WT     CTRL             0             EC0            4
    5          WT      PZA           400            EC99            1
    6          WT      PZA           400            EC99            2
    7          WT      RIF             2            EC99            1
    8          WT      RIF             2            EC99            2
    9          WT      INH             2            EC99            1
    10         WT      INH             2            EC99            2
    11         WT      BDQ           2.5            EC99            1
    12         WT      BDQ           2.5            EC99            2
5   3         RD1     CTRL             0             EC0            1
    4         RD1     CTRL             0             EC0            2
    5         RD1      PZA            60            EC50            1
    6         RD1      PZA            60            EC50            2
    7         RD1      RIF           0.1            EC50            1
    8         RD1      RIF           0.1            EC50            2
    9         RD1      INH          0.04            EC50            1
    10        RD1      INH          0.04            EC50            2
    11        RD1      BDQ          0.02            EC50            1
    12        RD1      BDQ          0.02            EC50            2
6   3         RD1     CTRL             0             EC0            3
    4         RD1     CTRL             0             EC0            4
    5         RD1      PZA           400            EC99            1
    6         RD1      PZA           400            EC99            2
    7         RD1      RIF             2            EC99            1
    8         RD1      RIF             2            EC99            2
    9         RD1      INH             2            EC99            1
    10        RD1      INH             2            EC99            2
    11        RD1      BDQ           2.5            EC99            1
    12        RD1      BDQ           2.5            EC99            2

# Segment, localise and track

In [9]:
mtb_channel = 0
gfp_channel = 1
manual_mtb_thresh_channel = 2
output_dirname = 'cpv3' #'cpv3_smaller_diam' #'cpv3'
# make output directory and subdirs 
os.makedirs(os.path.join(base_dir, f'labels/{output_dirname}/backup'), exist_ok = True)

# Just one position

In [6]:
row, column = 6, 3
acq_ID = (row, column)
log_progress(acq_ID, "Starting new acquisition")

# if info['Strain'] == 'UNI':
#     log_progress(acq_ID, "Skipping uninfected acquisition for now")
#     continue
# if acq_ID in already_processed_acq_IDs:
#     log_progress(acq_ID, "Skipping already processed")
#     continue
# if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_warea.h5')):
#     log_progress(acq_ID, "Skipping already processed")
#     continue

# process images using zarr
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
zarr_store = zarr.open(image_dir, mode='r')
images = zarr_store.images
# create a max projection
images = np.max(images, axis = 2)
# or just pick a z slice 
# images = zarr_store.images[:,:,0,...]

log_progress(acq_ID, "Images loaded and stacked")

# check if already segmented using m2 model
#     if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')):
#         continue
#     else:
log_progress(acq_ID, "Starting segmentation")

#     if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_seg_backup.h5')):
#         with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_seg_backup.h5'), 
#                                        'r', 
#                                        obj_type='obj_type_1'
#                                        ) as reader:
# #             writer.write_objects(objects)
#             # writer.write_tracks(tracks)
#             masks = reader.segmentation
#         log_progress(acq_ID, "Loaded previously calculated segmentation")
#     else:
# segment images from gfp channel only
masks = np.stack([segment(frame) 
                  for frame in tqdm(images[:,gfp_channel,...],  # segmenting the GFP channel 
                                    desc = 'Segmenting')])

log_progress(acq_ID, "Finished segmentation")

with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}_cpv3_mask_backup.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
#             writer.write_objects(objects)
        # writer.write_tracks(tracks)
        writer.write_segmentation(masks)
    
log_progress(acq_ID, "Saved out masks")  

log_progress(acq_ID, "Measuring Mtb area")       

# characterise Mtb growth using Otsu segmentation
# otsu_mtb = otsu_threshold(images[:,1,...]) # time consuming and non-deterministic when compared to hardcoded, could result in different thresholds for same image? 
# characterise Mtb growth using hardcoded threshold :S
manual_mtb_thresh = np.where(images[:,mtb_channel,...] >= Mtb_load_thresh, True, False)
log_progress(acq_ID, "Creating intensity image for localisation")  
# reshape intensity image to be gfp, rfp on last axis for regionprops
intensity_image = np.stack([images[:,0,...], 
                            images[:,1,...],  
#                                 otsu_mtb, 
                            manual_mtb_thresh], axis = -1)
log_progress(acq_ID, "Localising objects")  
# localise objects
objects = localise(masks, 
                   intensity_image, 
                   )
log_progress(acq_ID, "Filtering small objects")  
# filter out small objects
objects = [o for o in objects if o.properties['area'] > segment_size_thresh]

log_progress(acq_ID, "Adding infection labels to objects")  
# add label for infection
for obj in objects:
    obj.properties = ({"Infected": True} 
                        if obj.properties['mean_intensity'][manual_mtb_thresh_channel] > 0 # index 2 for manual mtb channel 
                        else {"Infected": False})
    obj.properties = ({"Mtb area px": obj.properties['mean_intensity'][manual_mtb_thresh_channel]*obj.properties['area']})

with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/backup/{row, column}_cpv3_objects_backup.h5'), 
                                           'w', 
                                           obj_type='obj_type_1'
                                           ) as writer:
                writer.write_objects(objects)
                # writer.write_tracks(tracks)

log_progress(acq_ID, "Beginning tracking")  
# track on upscaled config fn
tracks = track(objects, masks, config_fn, search_radius = 20)
log_progress(acq_ID, "Saving tracking")  
# save out 
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/backup/{row, column}_cpv3_tracks_backup.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
#             writer.write_objects(objects)
        writer.write_tracks(tracks)
        # writer.write_segmentation(masks)
# Log successful completion

with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}_cpv3_tracks.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
            writer.write_tracks(tracks)
            writer.write_segmentation(masks)
# Log successful completion
log_progress(acq_ID, "Processing completed successfully")

INFO:root:Position (6, 3): Starting new acquisition
INFO:root:Position (6, 3): Images loaded and stacked
INFO:root:Position (6, 3): Starting segmentation


Segmenting:   0%|          | 0/150 [00:00<?, ?it/s]

INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 58.96 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 46.14 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 49.70 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 42.40 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 35.84 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 39.00 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 41.89 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 39.16 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 41.99 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 42.98 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 40.81 sec
INFO:cellp

GLPK Integer Optimizer 5.0
10176 rows, 8119 columns, 11150 non-zeros
8119 integer variables, all of which are binary
Preprocessing...
5088 rows, 8119 columns, 11150 non-zeros
8119 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5088
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
5088 rows, 8119 columns, 11150 non-zeros
*     0: obj =   3.058517889e+04 inf =   0.000e+00 (2236)
Perturbing LP to avoid stalling [1495]...
Removing LP perturbation [2253]...
*  2253: obj =   1.367588464e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  2253: mip =     not found yet >=              -inf        (1; 0)
+  2253: >>>>>   1.367588464e+04 >=   1.367588464e+04   0.0% (1; 0)
+  2253: mip =   1.367588464e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOL

INFO:root:Position (6, 3): Saving tracking
[INFO][2024/04/15 05:53:57 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/backup/(6, 3)_cpv3_tracks_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/backup/(6, 3)_cpv3_tracks_backup.h5...
[INFO][2024/04/15 05:53:57 PM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/04/15 05:53:57 PM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/04/15 05:53:57 PM] Loading objects/obj_type_1 (10945, 5) (10945 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (10945, 5) (10945 filtered: None)
[INFO][2024/04/15 05:53:57 PM] Writing properties/obj_type_1/area (10945,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/area (10945,)
[INFO][2024/04/15 05:53:57 PM] Writing properties/obj_type_1/major_axis_length (10945,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/major_axis_length (10945,)
[INFO][2024/04/15 05:53:57 

# Iterate over all positions

In [10]:
# Inside your loop, use the log_progress function to log progress and errors
for (row, column), info in tqdm(assay_layout.iterrows(), desc='Progress through positions', total=len(assay_layout)):
    try:
        acq_ID = (row, column)
        log_progress(acq_ID, "Starting new acquisition")
        
        # if info['Strain'] == 'UNI':
        #     log_progress(acq_ID, "Skipping uninfected acquisition for now")
        #     continue
        # if acq_ID in already_processed_acq_IDs:
        #     log_progress(acq_ID, "Skipping already processed")
        #     continue
        if os.path.exists(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}.h5')):
            log_progress(acq_ID, "Skipping already processed")
            continue

        # process images using zarr
        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        zarr_store = zarr.open(image_dir, mode='r')
        images = zarr_store.images
        # create a max projection
        images = np.max(images, axis = 2)
        
        log_progress(acq_ID, "Images loaded and stacked")
        
        # check if already segmented using m2 model
        #     if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')):
        #         continue
        #     else:
        log_progress(acq_ID, "Starting segmentation")
        
        #     if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_seg_backup.h5')):
        #         with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_seg_backup.h5'), 
        #                                        'r', 
        #                                        obj_type='obj_type_1'
        #                                        ) as reader:
        # #             writer.write_objects(objects)
        #             # writer.write_tracks(tracks)
        #             masks = reader.segmentation
        #         log_progress(acq_ID, "Loaded previously calculated segmentation")
        #     else:
        # segment images from gfp channel only
        masks = np.stack([segment(frame) 
                          for frame in tqdm(images[:,gfp_channel,...],  # segmenting the GFP channel 
                                            desc = 'Segmenting')])
        
        log_progress(acq_ID, "Finished segmentation")
        
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}_cpv3_mask_backup.h5'), 
                                           'w', 
                                           obj_type='obj_type_1'
                                           ) as writer:
        #             writer.write_objects(objects)
                # writer.write_tracks(tracks)
                writer.write_segmentation(masks)
            
        log_progress(acq_ID, "Saved out masks")  
        
        log_progress(acq_ID, "Measuring Mtb area")       
        
        # characterise Mtb growth using Otsu segmentation
        # otsu_mtb = otsu_threshold(images[:,1,...]) # time consuming and non-deterministic when compared to hardcoded, could result in different thresholds for same image? 
        # characterise Mtb growth using hardcoded threshold :S
        manual_mtb_thresh = np.where(images[:,mtb_channel,...] >= Mtb_load_thresh, True, False)
        log_progress(acq_ID, "Creating intensity image for localisation")  
        # reshape intensity image to be gfp, rfp on last axis for regionprops
        intensity_image = np.stack([images[:,0,...], 
                                    images[:,1,...],  
        #                                 otsu_mtb, 
                                    manual_mtb_thresh], axis = -1)
        log_progress(acq_ID, "Localising objects")  
        # localise objects
        objects = localise(masks, 
                           intensity_image, 
                           )
        log_progress(acq_ID, "Filtering small objects")  
        # filter out small objects
        objects = [o for o in objects if o.properties['area'] > segment_size_thresh]
        
        log_progress(acq_ID, "Adding infection labels to objects")  
        # add label for infection
        for obj in objects:
            obj.properties = ({"Infected": True} 
                                if obj.properties['mean_intensity'][manual_mtb_thresh_channel] > 0 # index 2 for manual mtb channel 
                                else {"Infected": False})
            obj.properties = ({"Mtb area px": obj.properties['mean_intensity'][manual_mtb_thresh_channel]*obj.properties['area']})
        
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/backup/{row, column}_cpv3_objects_backup.h5'), 
                                                   'w', 
                                                   obj_type='obj_type_1'
                                                   ) as writer:
                        writer.write_objects(objects)
                        # writer.write_tracks(tracks)
        
        log_progress(acq_ID, "Beginning tracking")  
        # track on upscaled config fn
        tracks = track(objects, masks, config_fn, search_radius = 20)
        log_progress(acq_ID, "Saving tracking")  
        # save out 
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/backup/{row, column}_cpv3_tracks_backup.h5'), 
                                           'w', 
                                           obj_type='obj_type_1'
                                           ) as writer:
        #             writer.write_objects(objects)
                writer.write_tracks(tracks)
                # writer.write_segmentation(masks)
        # Log successful completion
        
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}.h5'), 
                                           'w', 
                                           obj_type='obj_type_1'
                                           ) as writer:
                    writer.write_tracks(tracks)
                    writer.write_segmentation(masks)
        # Log successful completion
        log_progress(acq_ID, "Processing completed successfully")

    except Exception as e:
        # Log errors
        log_progress(acq_ID, f"Processing failed: {str(e)}")

# You can also log information before and after the loop
logging.info("Processing completed")
hello nathan 
# Notify if required
notify.send_sms("Processing completed")


Progress through positions:   0%|          | 0/42 [00:00<?, ?it/s]

INFO:root:Position (3, 1): Starting new acquisition
INFO:root:Position (3, 1): Skipping already processed
INFO:root:Position (3, 2): Starting new acquisition
INFO:root:Position (3, 2): Skipping already processed
INFO:root:Position (3, 3): Starting new acquisition
INFO:root:Position (3, 3): Skipping already processed
INFO:root:Position (3, 4): Starting new acquisition
INFO:root:Position (3, 4): Skipping already processed
INFO:root:Position (3, 5): Starting new acquisition
INFO:root:Position (3, 5): Skipping already processed
INFO:root:Position (3, 6): Starting new acquisition
INFO:root:Position (3, 6): Skipping already processed
INFO:root:Position (3, 7): Starting new acquisition
INFO:root:Position (3, 7): Skipping already processed
INFO:root:Position (3, 8): Starting new acquisition
INFO:root:Position (3, 8): Skipping already processed
INFO:root:Position (3, 9): Starting new acquisition
INFO:root:Position (3, 9): Skipping already processed
INFO:root:Position (3, 10): Starting new acqui

Segmenting:   0%|          | 0/154 [00:00<?, ?it/s]

INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 44.74 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 49.69 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 58.37 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 59.64 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 60.56 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 66.47 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 48.70 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 56.54 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 47.85 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 45.76 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 49.35 sec
INFO:cellp

GLPK Integer Optimizer 5.0
23044 rows, 19655 columns, 27788 non-zeros
19655 integer variables, all of which are binary
Preprocessing...
11522 rows, 19655 columns, 27788 non-zeros
19655 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 11522
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
11522 rows, 19655 columns, 27788 non-zeros
*     0: obj =   6.685395222e+04 inf =   0.000e+00 (5279)
Perturbing LP to avoid stalling [2287]...
Removing LP perturbation [5332]...
*  5332: obj =   3.304066286e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  5332: mip =     not found yet >=              -inf        (1; 0)
+  5332: >>>>>   3.304066286e+04 >=   3.304066286e+04   0.0% (1; 0)
+  5332: mip =   3.304066286e+04 >=     tree is empty   0.0% (0; 1)
INTEGER O

...

Segmenting:   0%|          | 0/154 [00:00<?, ?it/s]

INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 37.94 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 46.06 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 48.23 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 49.51 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 45.28 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 44.11 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 47.97 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 48.54 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 52.84 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 49.14 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 46.30 sec
INFO:cellp

GLPK Integer Optimizer 5.0
22384 rows, 18970 columns, 26748 non-zeros
18970 integer variables, all of which are binary
Preprocessing...
11192 rows, 18970 columns, 26748 non-zeros
18970 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 11192
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
11192 rows, 18970 columns, 26748 non-zeros
*     0: obj =   6.533570263e+04 inf =   0.000e+00 (5186)
Perturbing LP to avoid stalling [2298]...
Removing LP perturbation [5298]...
*  5298: obj =   3.171134901e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  5298: mip =     not found yet >=              -inf        (1; 0)
+  5298: >>>>>   3.171134901e+04 >=   3.171134901e+04   0.0% (1; 0)
+  5298: mip =   3.171134901e+04 >=     tree is empty   0.0% (0; 1)
INTEGER O

[INFO][2024/05/02 04:06:07 AM] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session
INFO:root:Position (5, 8): Saving tracking
[INFO][2024/05/02 04:06:08 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/backup/(5, 8)_cpv3_tracks_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/backup/(5, 8)_cpv3_tracks_backup.h5...
[INFO][2024/05/02 04:06:08 AM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/05/02 04:06:08 AM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/05/02 04:06:08 AM] Loading objects/obj_type_1 (43238, 5) (43238 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (43238, 5) (43238 filtered: None)
[INFO][2024/05/02 04:06:09 AM] Writing properties/obj_type_1/area (43238,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/area (43238,)
[INFO][2024/05/02 04:06:09 AM] Writing properties/obj_type_1/major_axis_length (4

Update sms failed to send


In [11]:
import napari
import os, glob
from macrohet import dataio, tile, visualise, notify
import numpy as np
from macrohet import visualise
import os
import re
import numpy as np
import cv2
import btrack
import zarr
from skimage import io
import cv2
from tqdm.auto import tqdm

def trim_black_borders(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply binary thresholding to get black areas
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)  # find the largest contour
        x, y, w, h = cv2.boundingRect(cnt)
        image = image[y:y+h, x:x+w]
    return image

def crop_image_to_tiles(image, num_tiles_per_row, overlap_percentage):
    height, width, _ = image.shape

    # Calculate basic tile dimensions
    basic_tile_width = width // num_tiles_per_row
    basic_tile_height = height // num_tiles_per_row

    # Calculate the overlap in pixels
    overlap = int(overlap_percentage * basic_tile_width)

    # Calculate effective tile dimensions including overlap
    tile_width = basic_tile_width + overlap
    tile_height = basic_tile_height + overlap

    tiles = []
    for i in range(num_tiles_per_row):
        for j in range(num_tiles_per_row):
            # Calculate the position of the tile accounting for overlap
            left = max(0, i * basic_tile_width - overlap // 2)
            upper = max(0, j * basic_tile_height - overlap // 2)
            right = min(width, left + tile_width)
            lower = min(height, upper + tile_height)
            
            # Crop the tile
            crop = image[upper:lower, left:right]
            tiles.append(crop)

    return tiles

num_tiles_per_row = 3
overlap_percentage =0.1

tile_position_mapping_dictionary = {1:2, 
                                   2:7, 
                                   3:8, 
                                   4:3, 
                                   5:6, 
                                   6:9, 
                                   7:4, 
                                   8:5,
                                   9:1}

In [12]:
%%time
expt_ID = 'ND0003'

base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 36.4 s, sys: 891 ms, total: 37.3 s
Wall time: 38.3 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0301K1F1P1R1,Ok,r03c01f01p01-ch1sk1fk1fl1.tiff,3,1,1,1,0,1,1,...,0,0.135256499,0,2024-02-16T17:15:25.597+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
1,0301K1F1P1R2,Ok,r03c01f01p01-ch2sk1fk1fl1.tiff,3,1,1,1,0,2,1,...,0,0.135256499,0,2024-02-16T17:15:25.813+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
2,0301K1F1P2R1,Ok,r03c01f01p02-ch1sk1fk1fl1.tiff,3,1,1,2,0,1,1,...,2E-06,0.1352586,0,2024-02-16T17:15:26.157+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
3,0301K1F1P2R2,Ok,r03c01f01p02-ch2sk1fk1fl1.tiff,3,1,1,2,0,2,1,...,2E-06,0.1352586,0,2024-02-16T17:15:26.39+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
4,0301K1F1P3R1,Ok,r03c01f01p03-ch1sk1fk1fl1.tiff,3,1,1,3,0,1,1,...,4E-06,0.135260597,0,2024-02-16T17:15:26.733+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349267,0612K154F9P1R2,Ok,r06c12f09p01-ch2sk154fk1fl1.tiff,6,12,9,1,153,2,1,...,0,0.135008901,275402.773,2024-02-19T21:59:46.84+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349268,0612K154F9P2R1,Ok,r06c12f09p02-ch1sk154fk1fl1.tiff,6,12,9,2,153,1,1,...,2E-06,0.135010898,275402.773,2024-02-19T21:59:47.183+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349269,0612K154F9P2R2,Ok,r06c12f09p02-ch2sk154fk1fl1.tiff,6,12,9,2,153,2,1,...,2E-06,0.135010898,275402.773,2024-02-19T21:59:47.4+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349270,0612K154F9P3R1,Ok,r06c12f09p03-ch1sk154fk1fl1.tiff,6,12,9,3,153,1,1,...,4E-06,0.135012895,275402.773,2024-02-19T21:59:47.743+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."


In [13]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC  Replicate #
Row Column                                                           
3   1         UNI     CTRL             0             EC0            1
    2         UNI     CTRL             0             EC0            2
    3          WT     CTRL             0             EC0            1
    4          WT     CTRL             0             EC0            2
    5          WT      PZA            60            EC50            1
    6          WT      PZA            60            EC50            2
    7          WT      RIF           0.1            EC50            1
    8          WT      RIF           0.1            EC50            2
    9          WT      INH          0.04            EC50            1
    10         WT      INH          0.04            EC50            2
    11         WT      BDQ          0.02            EC50            1
    12         WT      BDQ          0.02            EC50            2
4   3          WT     CTRL             0             EC0            3
    4          WT     CTRL             0             EC0            4
    5          WT      PZA           400            EC99            1
    6          WT      PZA           400            EC99            2
    7          WT      RIF             2            EC99            1
    8          WT      RIF             2            EC99            2
    9          WT      INH             2            EC99            1
    10         WT      INH             2            EC99            2
    11         WT      BDQ           2.5            EC99            1
    12         WT      BDQ           2.5            EC99            2
5   3         RD1     CTRL             0             EC0            1
    4         RD1     CTRL             0             EC0            2
    5         RD1      PZA            60            EC50            1
    6         RD1      PZA            60            EC50            2
    7         RD1      RIF           0.1            EC50            1
    8         RD1      RIF           0.1            EC50            2
    9         RD1      INH          0.04            EC50            1
    10        RD1      INH          0.04            EC50            2
    11        RD1      BDQ          0.02            EC50            1
    12        RD1      BDQ          0.02            EC50            2
6   3         RD1     CTRL             0             EC0            3
    4         RD1     CTRL             0             EC0            4
    5         RD1      PZA           400            EC99            1
    6         RD1      PZA           400            EC99            2
    7         RD1      RIF             2            EC99            1
    8         RD1      RIF             2            EC99            2
    9         RD1      INH             2            EC99            1
    10        RD1      INH             2            EC99            2
    11        RD1      BDQ           2.5            EC99            1
    12        RD1      BDQ           2.5            EC99            2

### Iterate and save out 

In [14]:
for acq_ID, info in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    # if acq_ID[0] != 6:
    #     continue
    # if acq_ID == (6, 3):
    #     continue
    output_basedir = os.path.join(base_dir, F'postfix/labelled_tracks_final_frame/{acq_ID}')
    if os.path.exists(output_basedir): 
        continue    
    try:
        os.makedirs(output_basedir, exist_ok=True)  # Make sure the base directory exists
    
        cropped_metadata = metadata[(metadata['Row'] == str(acq_ID[0]))
                                    & (metadata['Col'] == str(acq_ID[1]))
                                    & (metadata['TimepointID'] == str(metadata['TimepointID'].astype(int).max()))
                                    & (metadata['ChannelID'] == '2')
                                    & (metadata['PlaneID'] == '1')
                                    ]
        
        
        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        zarr_group = zarr.open(image_dir, mode='r')
        
        images = zarr_group.images[-1,:,0,...]
        images.shape
        
        viewer = napari.Viewer(title = f'{expt_ID, acq_ID}')
        
        viewer.add_image(images,
                         channel_axis = 0, 
                         # # scale = napari_scale, 
                         contrast_limits=[[280, 1000],[0,3000]])
        
        with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/{expt_ID}/labels/cpv3/{acq_ID}.h5'), 
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                        segmentation = reader.segmentation
                        tracks = reader.tracks
        
        last_frame_tracks = [t for t in tracks if t.in_frame(149) and len(t) >= 70]
        coords = [(int(t.y[-1]*5.04), int(t.x[-1]*5.04)) for t in last_frame_tracks]
        # Assuming last_frame_tracks is a list of objects with attributes ID, x, and y
        id_xy_dict = [{'ID':t.ID, 
                      'x':int(t.x[-1]*5.04),
                      'y':int(t.y[-1]*5.04)} for t in last_frame_tracks]
        
        ### JUST IDs
        text_parameters = {
            'string': '{ID}',
            'size': 9,
            'color': 'white',
            'anchor': 'center',
            # 'translation': [-3, 0],
        }
        viewer.add_points(coords, size = 150, face_color='transparent', edge_color='transparent', edge_width=0.1, 
                          properties = id_xy_dict,
                          text = text_parameters)
        output_fn = f'{acq_ID}_{expt_ID}_t-1_IDs.png'
        output_path = os.path.join(output_basedir, 'tiled', output_fn)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Make sure the directory exists
        screen_shot = trim_black_borders(viewer.screenshot())
        io.imsave(output_path, screen_shot)
        tiles = crop_image_to_tiles(screen_shot, 3, 0.1)
        for i, tile in enumerate(tiles): 
            FieldID = tile_position_mapping_dictionary[i+1]
            tile_fn = cropped_metadata[cropped_metadata['FieldID'] == str(FieldID)]['URL'].iloc[0]
            directory, old_filename = os.path.split(output_path)
            new_filename = f"{tile_fn}_{old_filename}"
            new_directory = directory.replace('tiled', 'untiled')    
            new_path = os.path.join(new_directory, new_filename)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)  # Make sure the directory exists
            io.imsave(new_path, tile)
        del viewer.layers['Points']
        
        ### IDs and COORDS
        text_parameters = {
            'string': '{ID}\n{x},{y}',
            'size': 9,
            'color': 'white',
            'anchor': 'center',
            # 'translation': [-3, 0],
        }
        viewer.add_points(coords, size = 150, face_color='transparent', edge_color='transparent', edge_width=0.1, 
                          properties = id_xy_dict,
                          text = text_parameters)
        output_fn = f'{acq_ID}_{expt_ID}_t-1_IDs_xy.png'
        output_path = os.path.join(output_basedir, 'tiled/with_xy_coords', output_fn)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Make sure the directory exists
        screen_shot = trim_black_borders(viewer.screenshot())
        io.imsave(output_path, screen_shot)
        tiles = crop_image_to_tiles(screen_shot, 3, 0.1)
        for i, tile in enumerate(tiles): 
            FieldID = tile_position_mapping_dictionary[i+1]
            tile_fn = cropped_metadata[cropped_metadata['FieldID'] == str(FieldID)]['URL'].iloc[0]
            directory, old_filename = os.path.split(output_path)
            new_filename = f"{tile_fn}_{old_filename}"
            new_directory = directory.replace('tiled', 'untiled')    
            new_path = os.path.join(new_directory, new_filename)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)  # Make sure the directory exists
            io.imsave(new_path, tile)
        del viewer.layers['Points']
        
        ### IDs with MASKS
        text_parameters = {
            'string': '{ID}',
            'size': 9,
            'color': 'white',
            'anchor': 'center',
            # 'translation': [-3, 0],
        }
        viewer.add_points(coords, size = 150, face_color='transparent', edge_color='transparent', edge_width=0.1, 
                          properties = id_xy_dict,
                          text = text_parameters)
        viewer.add_labels(segmentation[-1])
        viewer.layers['Labels'].contour = 10
        output_fn = f'{acq_ID}_{expt_ID}_t-1_IDs_masks.png'
        output_path = os.path.join(output_basedir, 'tiled/with_masks', output_fn)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Make sure the directory exists
        screen_shot = trim_black_borders(viewer.screenshot())
        io.imsave(output_path, screen_shot)
        tiles = crop_image_to_tiles(screen_shot, 3, 0.1)
        for i, tile in enumerate(tiles): 
            FieldID = tile_position_mapping_dictionary[i+1]
            tile_fn = cropped_metadata[cropped_metadata['FieldID'] == str(FieldID)]['URL'].iloc[0]
            directory, old_filename = os.path.split(output_path)
            new_filename = f"{tile_fn}_{old_filename}"
            new_directory = directory.replace('tiled', 'untiled')    
            new_path = os.path.join(new_directory, new_filename)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)  # Make sure the directory exists
            io.imsave(new_path, tile)
        del viewer.layers['Points']
        
        ### IDs and COORDS and masks
        text_parameters = {
            'string': '{ID}\n{x},{y}',
            'size': 9,
            'color': 'white',
            'anchor': 'center',
            # 'translation': [-3, 0],
        }
        viewer.add_points(coords, size = 150, face_color='transparent', edge_color='transparent', edge_width=0.1, 
                          properties = id_xy_dict,
                          text = text_parameters)
        # viewer.add_labels(segmentation[-1])
        # viewer.layers['Labels'].contour = 10
        output_fn = f'{acq_ID}_{expt_ID}_t-1_IDs_xy_masks.png'
        output_path = os.path.join(output_basedir, 'tiled/with_xy_coords_masks', output_fn)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Make sure the directory exists
        screen_shot = trim_black_borders(viewer.screenshot())
        io.imsave(output_path, screen_shot)
        tiles = crop_image_to_tiles(screen_shot, 3, 0.1)
        for i, tile in enumerate(tiles): 
            FieldID = tile_position_mapping_dictionary[i+1]
            tile_fn = cropped_metadata[cropped_metadata['FieldID'] == str(FieldID)]['URL'].iloc[0]
            directory, old_filename = os.path.split(output_path)
            new_filename = f"{tile_fn}_{old_filename}"
            new_directory = directory.replace('tiled', 'untiled')    
            new_path = os.path.join(new_directory, new_filename)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)  # Make sure the directory exists
            io.imsave(new_path, tile)
        del viewer.layers['Points']
        del viewer.layers['Labels']
        viewer.close()
    except:
        print(acq_ID)

  0%|          | 0/42 [00:00<?, ?it/s]